# Carga de movimentação dos correios com arquivo personalizado completo 

In [9]:
#importar bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import pyodbc as db
import re
import csv
from decimal import Decimal

In [10]:
#Estabelece conexão com o banco de dados
database = db.connect('DRIVER={MySQL ODBC 8.0 ANSI Driver};SERVER=localhost;DATABASE=sea_lake; Uid=root;Pwd=@Fpolis18;OPTION=3;')
cursor = database.cursor()
#Criar os inserts para cada tipo de registro
query_cabecalho = """INSERT INTO correio_completo_cabecalho (cnpj, razao_social, codigo_cliente, numero_contrato, dr_contrato, codigo_centro_custo, centro_custo, numero_fatura, dr_fatura, status_fatura, inicio_ciclo_faturamento, fim_ciclo_faturamento, data_emissao, data_vencimento, data_pagamento, data_cancelamento, data_fechamento) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""
query_movimento = """INSERT INTO correio_completo_movimento (numero_cartao_postagem, titular_cartao_postagem, data_postagem, codigo_servico, servico, quantidade_itens, peso, valor_unitario_servico, valor_servico, numero_documento, etiqueta, valor_declarado, valor_desconto, codigo_servico_adicional, unidade_postagem, cep_origem, municipio_cep_origem, uf_cep_origem, cep_destino, muncipio_cep_destino, uf_cep_destino) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""
query_credito = """INSERT INTO correio_completo_credito (descricao_credito, valor_credito) VALUES (?, ?)"""
query_debito = """INSERT INTO correio_completo_debito (descricao_debito, valor_debito) VALUES (?, ?)"""
query_rodape = """INSERT INTO correio_completo_rodape (valor_liquido_fatura, valor_bruto_fatura, lancamentos_debitos, lancamentos_creditos, complementacao_financeira, restituicao_complmentacao_financeira, servicos_meses_anteriores, servicos_ciclo_atual, encargos, descontos, quantidade_registros) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""

In [11]:
#file = "Correio-2017-12-Completo.csv" - ok
#file = "Correio-2018-01-Completo.csv" - ok 
#file = "Correio-2018-02-Completo.csv" - ok
#file = "Correio-2018-03-Completo.csv" - ok
#file = "Correio-2018-04-Completo.csv" - ok
#file = "Correio-2018-05-Completo.csv" - ok
#file = "Correio-2018-06-Completo.csv" - ok
#filenew = ("novo_" + file)

In [12]:
with open(file, 'r', encoding= "utf8") as csvfile:
    arquivo = csv.reader(csvfile, delimiter=';', quotechar='|')
    count = 0
    for row in arquivo:
        count = count + 1
    linhas = count
    print (linhas)

151526


In [13]:
with open(file, 'r', encoding= "utf8") as csvfile:
    arquivo = csv.reader(csvfile, delimiter=';', quotechar='|')
    count = 0
    for row in arquivo:
        count = count + 1
        campo_a = row[0:1]
        if count == 1:
            continue #desprezar header do cabeçalho
        else:
            if count == 2:
                #print ("Cabeçalho")
                #CNPJ
                cnpj = str(row[0]) #A
                #Razão Social
                razao_social = str(row[1]) #B
                #Código do cliente
                codigo_cliente = str(row[2]) #C
                #Número do contrato
                numero_contrato = str(row[3]) #D
                #DR do número do contrato
                dr_contrato = str(row[4]) #E
                #Código do centro de custos
                codigo_centro_custo = str(row[5]) #F
                #Centro de custos
                centro_custo = str(row[6]) #G
                #Número da fatura
                numero_fatura = str(row[7]) #H 
                #DR do número da fatura
                dr_fatura = str(row[8]) #I
                #Status da fatura
                status_fatura = str(row[9]) #J
                #Início do ciclo de faturamento
                inicio_ciclo_faturamento = str(row[10]) #K
                #Fim do ciclo de faturamento
                fim_ciclo_faturamento = str(row[11]) #L
                #Data de emissão
                data_emissao = str(row[12]) #M
                #Data de vencimento
                data_vencimento = str(row[13]) #N
                #Data de pagamento
                data_pagamento = str(row[14]) #O
                #Data de cancelamento
                data_cancelamento = str(row[15]) #P
                #Data de fechamento
                data_fechamento = str(row[16]) #Q
                #Insert
                values = (cnpj, razao_social, codigo_cliente, numero_contrato, dr_contrato, codigo_centro_custo, centro_custo, numero_fatura, dr_fatura, status_fatura, inicio_ciclo_faturamento, fim_ciclo_faturamento, data_emissao, data_vencimento, data_pagamento, data_cancelamento, data_fechamento) 
                cursor.execute(query_cabecalho, values)
            else:
                if count == linhas:
                    #print("Rodapé")
                    #Valor líquido da fatura
                    valor_liquido_fatura = str(row[0]) #A -Decimal
                    for char in 'R$. ':
                        valor_liquido_fatura = valor_liquido_fatura.replace(char,'')
                    for char in ',':
                        valor_liquido_fatura = valor_liquido_fatura.replace(char,'.')
                    valor_liquido_fatura = Decimal(valor_liquido_fatura[0:20])
                    #Valor bruto da fatura
                    valor_bruto_fatura = str(row[1]) #B - Decimal
                    for char in 'R$. ':
                        valor_bruto_fatura = valor_bruto_fatura.replace(char,'')
                    for char in ',':
                        valor_bruto_fatura = valor_bruto_fatura.replace(char,'.')
                    valor_bruto_fatura = Decimal(valor_bruto_fatura[0:20])                    
                    #Lançamentos dos débitos
                    lancamentos_debitos = str(row[2]) #C - Decimal
                    for char in 'R$. ':
                        lancamentos_debitos = lancamentos_debitos.replace(char,'')
                    for char in ',':
                        lancamentos_debitos = lancamentos_debitos.replace(char,'.')
                    lancamentos_debitos = Decimal(lancamentos_debitos[0:20])                      
                    #Lançamentos dos créditos                 
                    lancamentos_creditos = str(row[3]) #D - Decimal
                    if (lancamentos_creditos == ' '):
                        for char in 'R$. ':
                            lancamentos_creditos = lancamentos_creditos.replace(char,'')
                        for char in ',':
                            lancamentos_creditos = lancamentos_creditos.replace(char,'.')
                        lancamentos_creditos = Decimal(lancamentos_creditos[0:20])                      
                    else:
                        lancamentos_creditos = 0
                    #Complementação financeira
                    complementacao_financeira  = str(row[4]) #E - Vazio
                    #Restituição da complementação financeira
                    restituicao_complmentacao_financeira = str(row[5]) #F - Vazio
                    #Serviços de meses anteriores
                    servicos_meses_anteriores = str(row[6]) #G - Decimal
                    print(servicos_meses_anteriores)
                    if (servicos_meses_anteriores == ' '):
                        for char in 'R$. ':
                            servicos_meses_anteriores = servicos_meses_anteriores.replace(char,'')
                        for char in ',':
                            servicos_meses_anteriores = servicos_meses_anteriores.replace(char,'.')
                        servicos_meses_anteriores = Decimal(servicos_meses_anteriores[0:20])
                    else:
                        servicos_meses_anteriores = 0
                    #Serviços do ciclo atual
                    servicos_ciclo_atual = str(row[7]) #H - Decimal
                    for char in 'R$. ':
                        servicos_ciclo_atual = servicos_ciclo_atual.replace(char,'')
                    for char in ',':
                        servicos_ciclo_atual = servicos_ciclo_atual.replace(char,'.')
                    servicos_ciclo_atual = Decimal(servicos_ciclo_atual[0:20])                      
                    #Encargos
                    encargos = str(row[8]) #I - Decimal
                    #Descontos
                    descontos = str(row[9]) #J - Decimal
                    for char in 'R$. ':
                        descontos = descontos.replace(char,'')
                    for char in ',':
                        descontos = descontos.replace(char,'.')
                    descontos = Decimal(descontos[0:20])                       
                    #Quantidades de registros 
                    quantidade_registros = int(row[10]) #K - Inteiro
                    #Insert
                    values = (valor_liquido_fatura, valor_bruto_fatura, lancamentos_debitos, lancamentos_creditos, complementacao_financeira, restituicao_complmentacao_financeira, servicos_meses_anteriores, servicos_ciclo_atual, encargos, descontos, quantidade_registros)
                    cursor.execute(query_rodape, values)
                else:
                    tipo = str(campo_a)
                    if (tipo[2] == '0'):
                        #print("Movimento")
                        #Numero do cartão de postagem
                        numero_cartao_postagem = str(row[0]) #A
                        #Titular do cartão de postagem
                        titular_cartao_postagem = str(row[1]) #B
                        #Data da postagem
                        data_postagem = str(row[2]) #C
                        #Código do serviço
                        codigo_servico = str(row[3]) #D
                        #Serviço
                        servico = str(row[4]) #E
                        #Quantidade de itens
                        quantidade_itens = int(row[5]) #F
                        #Peso
                        peso = int(row[6]) #G - Peso em gramas
                        #Valor unitário do serviço
                        valor_unitario_servico = str(row[7]) #H - Decimal
                        for char in 'R$. ':
                            valor_unitario_servico = valor_unitario_servico.replace(char,'')
                        for char in ',':
                            valor_unitario_servico = valor_unitario_servico.replace(char,'.')
                        valor_unitario_servico = Decimal(valor_unitario_servico[0:20])  
                        #Valor do serviço
                        valor_servico = str(row[8]) #I - Decimal
                        for char in 'R$. ':
                            valor_servico = valor_servico.replace(char,'')
                        for char in ',':
                            valor_servico = valor_servico.replace(char,'.')
                        valor_servico = Decimal(valor_servico[0:20])                          
                        #Número do documento
                        numero_documento = str(row[9]) #J
                        #Etiqueta
                        etiqueta = str(row[10]) #K
                        #Valor declarado
                        valor_declarado = str(row[11]) #L - Decimal
                        for char in 'R$. ':
                            valor_declarado = valor_declarado.replace(char,'')
                        for char in ',':
                            valor_declarado = valor_declarado.replace(char,'.')
                        valor_declarado = Decimal(valor_declarado[0:20])                          
                        #Valor do desconto
                        valor_desconto = str(row[12]) #M - Decimal
                        for char in 'R$. ':
                            valor_desconto = valor_desconto.replace(char,'')
                        for char in ',':
                            valor_desconto = valor_desconto.replace(char,'.')
                        valor_desconto = Decimal(valor_desconto[0:20])   
                        #Código do serviço adicional
                        codigo_servico_adicional = str(row[13]) #N
                        #Unidade de postagem
                        unidade_postagem = str(row[14]) #O
                        #CEP da unidade de origem
                        cep_origem = str(row[15]) #P
                        #Muncípio do CEP da origem
                        municipio_cep_origem = str(row[16]) #Q
                        #UF do CEP da origem
                        uf_cep_origem = str(row[17]) #R
                        #CEP do destino
                        cep_destino = str(row[18]) #S
                        #Município do CEP de destino
                        muncipio_cep_destino = str(row[19]) #T
                        #UF do CEP de destino
                        uf_cep_destino = str(row[20]) #U
                        #Insert
                        values = (numero_cartao_postagem, titular_cartao_postagem, data_postagem, codigo_servico, servico, quantidade_itens, peso, valor_unitario_servico, valor_servico, numero_documento, etiqueta, valor_declarado, valor_desconto, codigo_servico_adicional, unidade_postagem, cep_origem, municipio_cep_origem, uf_cep_origem, cep_destino, muncipio_cep_destino, uf_cep_destino)
                        cursor.execute(query_movimento, values)
                    else:
                        if (tipo[2] == 'C' or tipo[2] == 'I'):
                            #print ("Crédito")
                            #Descrição do crédito
                            descricao_credito = str(row[0]) #A
                            #Valor do crédito
                            valor_credito = str(row[1]) #B
                            for char in 'R$. ':
                                valor_credito = valor_credito.replace(char,'')
                            for char in ',':
                                valor_credito = valor_credito.replace(char,'.')
                            valor_credito = Decimal(valor_credito[0:20])
                            #Insert
                            values = (descricao_credito, valor_credito)
                            cursor.execute(query_credito, values)
                        else:
                            if (tipo[2] == 'D' or tipo[2:3] == 'VSC' or tipo[2] == 'S'):
                                #print ("Débito")
                                #Descrção do débito
                                descricao_debito = str(row[0]) #A
                                #Valor do débito
                                valor_debito = str(row[1]) #B - Decimal
                                for char in 'R$. ':
                                    valor_debito = valor_debito.replace(char,'')
                                for char in ',':
                                    valor_debito = valor_debito.replace(char,'.')
                                valor_debito = Decimal(valor_debito[0:20])
                                #Insert
                                values = (descricao_debito, valor_debito)
                                cursor.execute(query_debito, values)

R$ 89,21


In [14]:
cursor.close()
database.commit()
database.close()
#file.close()

In [300]:
#

create table correio_completo_cabecalho 
(cnpj varchar(14), 
 razao_social varchar(100), 
 codigo_cliente varchar(12), 
 numero_contrato varchar(12), 
 dr_contrato varchar(2), 
 codigo_centro_custo varchar(12), 
 centro_custo varchar(100), 
 numero_fatura varchar(12), 
 dr_fatura varchar(2), 
 status_fatura varchar(20), 
 inicio_ciclo_faturamento varchar(10), 
 fim_ciclo_faturamento varchar(10), 
 data_emissao varchar(10), 
 data_vencimento varchar(10), 
 data_pagamento varchar(10), 
 data_cancelamento varchar(10), 
 data_fechamento varchar(10)
);

create table correio_completo_rodape 
(valor_liquido_fatura decimal(12,2), 
 valor_bruto_fatura decimal(12,2), 
 lancamentos_debitos decimal(12,2), 
 lancamentos_creditos decimal(12,2), 
 complementacao_financeira varchar(20), 
 restituicao_complmentacao_financeira varchar(20), 
 servicos_meses_anteriores decimal(12,2), 
 servicos_ciclo_atual decimal(12,2), 
 encargos varchar(20), 
 descontos decimal(12,2), 
 quantidade_registros integer
);

create table correio_completo_movimento 
(numero_cartao_postagem varchar(12), 
 titular_cartao_postagem varchar(100), 
 data_postagem varchar(10), 
 codigo_servico varchar(12), 
 servico varchar(100), 
 quantidade_itens integer, 
 peso integer, 
 valor_unitario_servico decimal(12,2), 
 valor_servico decimal(12,2), 
 numero_documento varchar(10), 
 etiqueta varchar(13), 
 valor_declarado decimal(12,2), 
 valor_desconto decimal(12,2), 
 codigo_servico_adicional varchar(12), 
 unidade_postagem varchar(100), 
 cep_origem varchar(8), 
 municipio_cep_origem varchar(100), 
 uf_cep_origem varchar(2), 
 cep_destino varchar(8), 
 municipio_cep_destino varchar(100), 
 uf_cep_destino varchar(2)
 );
 
 
create table correio_completo_credito
(descricao_credito varchar(100), 
 valor_credito decimal(12,2)
);

create table correio_completo_debito
(descricao_debito varchar(100), 
 valor_debito decimal(12,2)
);